In [1]:
import pandas as pd
import altair as alt

In [1]:
assays_long = pd.read_csv('assays_long.csv')

NameError: name 'pd' is not defined

In [ ]:
assays_long['outcome'].unique()

# Proportion stacked bar plot:

In [42]:
###Count up conditions using groupby:
numActive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Active').any())).values.sum()
numInactive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Inactive').any())).values.sum()

numBoth = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: ((grouped_df['outcome'].str.contains('Inactive').any()) & 
                        (grouped_df['outcome'].str.contains('Active').any()))).values.sum()

numNone = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        ~(grouped_df['outcome'].str.contains('Active').any()))).values.sum()

In [43]:
###Or count up conditions the boring way:
y=0
n=0
b=0
neither=0
instance_ids = assays_long.drop_duplicates(['instance_id', 'pref_name'])['instance_id']
pref_names = assays_long.drop_duplicates(['instance_id', 'pref_name'])['pref_name']
for i, name in zip(instance_ids, pref_names):
    mask = (assays_long['pref_name']==name) & (assays_long['instance_id']==i)
    group = assays_long[mask]
    outcomes = list(group['outcome'])
    if ('Active' in outcomes) and ('Inactive' in outcomes):
        b+=1
    elif ('Active' in outcomes):
        y+=1
    elif ('Inactive' in outcomes):
        n+=1
    else:
        neither+=1
        

In [44]:
assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        ~(grouped_df['outcome'].str.contains('Active').any()))).values.sum()

1443

In [45]:
#Verify both ways are the same:
print(f'Active: {y}')
print(f'Inactive: {n}')
print(f'Both: {b}')
print(f'None: {neither}')

Active: 708
Inactive: 109
Both: 24
None: 1443


In [46]:
#Both ways are the same:
print(f'Active: {numActive-numBoth}')
print(f'Inactive: {numInactive-numBoth}')
print(f'Both: {numBoth}')
print(f'None: {numNone}')

Active: 708
Inactive: 109
Both: 24
None: 1443


In [78]:
df = pd.DataFrame(columns=['results','activity', 'number'])
df.loc[0]=['Results', 'Active', numActive]
df.loc[1]=['Results', 'Inactive', numInactive]
df.loc[2]=['Results', 'Both', numBoth]

###Remember - inconclusive or unspecified must be added to all the predictions that had NO records.
#without the 'unknown' records:
#df.loc[3]=['Results', 'Unknown', numNone ]
#including all absent records:
number_of_absent_records = 10000 - sum([df.iloc[0]['number'], df.iloc[1]['number']])
df.loc[3] = ['Results', 'Unknown', number_of_absent_records]

df['proportion'] = df['number']/df['number'].sum()
df

,results,activity,number,proportion
0,Results,Active,732,0.0730247
1,Results,Inactive,133,0.0132682
2,Results,Both,24,0.00239425
3,Results,Unknown,9135,0.911313


In [104]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='All assays'),#,sort='descending'),
    y=alt.Y('results', title=''),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
)#.add_selection(
 #   selector
#)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Assays with activity evidence'),#,sort='descending'),
    y=alt.Y('results', title=''),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
)
#.add_selection(
#    selector
#)

chart_one&chart_two


alt.VConcatChart(...)

In [105]:
####Interactive version:
# brush = alt.selection(type='interval', encodings=['x'])

# base = alt.Chart(df).transform_joinaggregate(
#         Total='sum(number)'
#     ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
#     #x='number',
#     x='PercentOfTotal:Q',
#     y='results',
#     color='activity')

# upper = base.encode(
#     alt.X('PercentOfTotal:Q', scale=alt.Scale(domain=brush)))

# lower = base.add_selection(brush)

# lower&upper 


# Target type barplot:


In [106]:
out = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        (grouped_df['outcome'].str.contains('Active').any())))

positive_targets=[i[1] for i, v in zip(out.index, out.values) if v==True]
all_pos_targets = pd.DataFrame(out[out==True]).reset_index().drop([0],axis=1)
pos_targets_gb= all_pos_targets.groupby(['pref_name']).count().reset_index().sort_values(by='instance_id', ascending=False)
pos_targets_gb.columns=['pref_name', 'count']



In [109]:
##Do the count aggregation within altair. 
##Can't figure out how to get it ordered by 'count' for some reason.
chart = alt.Chart(all_pos_targets).mark_bar()

# chart = chart.encode(
#     x = alt.X('pref_name', sort='-y'),
#     y = alt.Y('count()'), 
# )#.configure_axisBottom(labelAngle=-45)


chart = alt.Chart(out_df).mark_bar()
chart = ch.encode(
    x = alt.X('pref_name', sort='-y', title='Target name'),
    y = alt.Y('count()', title='Count'),
    color=alt.Color('activity'),
    order=alt.Order('activity', sort='ascending'),
).properties(height=200,width=700)



In [110]:
(chart_one.properties(width=700)&chart_two.properties(width=700)&chart).configure_axisBottom(labelAngle=-45)

alt.VConcatChart(...)

# All together now:

In [57]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y=alt.Y('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).properties(height=30, width=550)
#.add_selection(
 #   selector
#).properties(height=30, width=550)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Hi'),#,sort='descending'),
    y=alt.Y('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).properties(height=30, width=550)
#.add_selection(
 #   selector
#).properties(height=30, width=550)

chart = alt.Chart(all_pos_targets).mark_bar()

chart = chart.encode(
    x = alt.X('pref_name', sort='-y', title='Target preferred name'),
    y = alt.Y('count()'), 
).properties(height=200,width=700)

(chart_one&chart_two&chart
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_title()

alt.VConcatChart(...)

In [58]:
def parse_group(group):
    inactive = group['outcome'].str.contains('Inactive').any()
    active = group['outcome'].str.contains('Active').any()
    if inactive and active:
        return 'Both'
    elif active and ~inactive:
        return 'Active'
    elif inactive and ~active:
        return 'Inactive'
    else:
        return False
    
    
out = assays_long.groupby(['instance_id', 'pref_name']).apply(parse_group)
out_df= pd.DataFrame(out[out!=False]).reset_index().drop('instance_id',axis=1)
out_df.columns=['pref_name', 'activity']

In [59]:
out_df

,pref_name,activity
0,Tyrosine-protein kinase JAK2,Inactive
1,Vascular endothelial growth factor receptor 2,Inactive
2,Fibroblast growth factor receptor 2,Both
3,Fibroblast growth factor receptor 3,Inactive
4,Tyrosine-protein kinase SRC,Inactive
...,...,...
836,Vascular endothelial growth factor receptor 2,Inactive
837,Serine/threonine-protein kinase PIM2,Inactive
838,Tyrosine-protein kinase JAK2,Inactive
839,Vascular endothelial growth factor receptor 2,Both


In [112]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    y=alt.Y('results', title='All records',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).properties(height=30, width=750)
#.add_selection(
#    selector
#).properties(height=30, width=550)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    y=alt.Y('results', title='Records with evidence',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).properties(height=30, width=750)
#.add_selection(
#    selector
#).properties(height=30, width=550)








ch = alt.Chart(out_df).mark_bar()
ch = ch.encode(
    x = alt.X('pref_name', sort='-y', title='Target preferred name'),
    y = alt.Y('count()'),
    color=alt.Color('activity'),
    order=alt.Order('activity', sort='ascending'),
).properties(height=200,width=750)


(chart_one&chart_two&ch
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_axisY(
    titleAngle=0,
    titleAlign="left",
    titleY=-10,
    titleX=-10,
    titleFontSize=16,
)


alt.VConcatChart(...)

In [75]:


chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    y=alt.Y('PercentOfTotal:Q', title='All assays'),#,sort='descending'),
    x=alt.X('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title=''),
    order=alt.Order('activity', sort='ascending')
).properties(height=500, width=30)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    y=alt.Y('PercentOfTotal:Q', title='Existing evidence',),#,sort='descending'),
    x=alt.X('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title=''),
    order=alt.Order('activity', sort='ascending')
).properties(height=500, width=30)








ch = alt.Chart(out_df).mark_bar()
ch = ch.encode(
    y = alt.Y('pref_name', sort='-x', title='Target occurrence'),
    x = alt.X('count()', title='Count'),
    color=alt.Color('activity'),
    order=alt.Order('activity', sort='ascending'),
).properties(height=500,width=300)


(chart_one|chart_two|ch
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_axisY(
    titleAngle=0,
    titleAlign="left",
    titleY=-10,
    titleX=-20,
    titleFontSize=16,
).configure_legend(orient='bottom',labelFontSize=22)
#top right' is not one of ['none', 'left', 'right', 'top', 
#'bottom', 'top-left', 'top-right', 'bottom-left', 'bottom-right']

alt.HConcatChart(...)

In [68]:
df['number'].iloc[:3]/df['number'].iloc[:3].sum()

0     0.823397
1     0.149606
2    0.0269966
Name: number, dtype: object

In [69]:
df

,results,activity,number,proportion
0,Results,Active,732,0.0730247
1,Results,Inactive,133,0.0132682
2,Results,Both,24,0.00239425
3,Results,Unknown,9135,0.911313


In [70]:
1-0.618782

0.38121799999999995

In [71]:
df

,results,activity,number,proportion
0,Results,Active,732,0.0730247
1,Results,Inactive,133,0.0132682
2,Results,Both,24,0.00239425
3,Results,Unknown,9135,0.911313


In [72]:
10000-9135

865

In [114]:
865*100/10000


8.65